<a href="https://colab.research.google.com/github/superishi/Cybersecurity-Flow-Anomaly-Detection/blob/main/WeekendAnomalyClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creating extra columns to be used for filtering
### time aggregated for every 3 hours, model focuses on 09:00 - 17:00 time period

In [3]:
def extract_day(time_string):
  day_string = time_string[:10]
  return day_string

In [4]:
def extract_time(time_string):
  time_string = time_string[-8:]
  return time_string

In [7]:
origData = pd.read_csv("/content/drive/MyDrive/DATA3001 Cyber/by3hours.csv", header = 0)
#print(origData)
origData['edit_date'] = origData['time'] 
origData['edit_date'] = origData['edit_date'].apply(extract_day)

origData['edit_time'] = origData['time']
origData['edit_time'] = origData['edit_time'].apply(extract_time)

weekends = ["2021-08-28", "2021-08-29", "2021-09-04", "2021-09-05"]

origData = origData.drop(origData[~origData.edit_date.isin(weekends)].index )


remove_time_set = ["00:00:00", "03:00:00", "06:00:00", "18:00:00", "21:00:00"]

origData = origData.drop(origData[origData.edit_time.isin(remove_time_set)].index )

print(origData)

                      time   device_mac  ...   edit_date  edit_time
38     2021-08-28 09:00:00  21:fa:27:56  ...  2021-08-28   09:00:00
39     2021-08-28 12:00:00  21:fa:27:56  ...  2021-08-28   12:00:00
40     2021-08-28 15:00:00  21:fa:27:56  ...  2021-08-28   15:00:00
46     2021-08-29 09:00:00  21:fa:27:56  ...  2021-08-29   09:00:00
47     2021-08-29 12:00:00  21:fa:27:56  ...  2021-08-29   12:00:00
...                    ...          ...  ...         ...        ...
85174  2021-09-04 12:00:00  ee:dd:4f:cd  ...  2021-09-04   12:00:00
85175  2021-09-04 15:00:00  ee:dd:4f:cd  ...  2021-09-04   15:00:00
85181  2021-09-05 09:00:00  ee:dd:4f:cd  ...  2021-09-05   09:00:00
85182  2021-09-05 12:00:00  ee:dd:4f:cd  ...  2021-09-05   12:00:00
85183  2021-09-05 15:00:00  ee:dd:4f:cd  ...  2021-09-05   15:00:00

[9048 rows x 8 columns]


In [8]:
excluded_devices = ["bd:1c:dc:fb", "ac:f2:14:98", "ac:f4:65:4a", "bd:0d:af:4f"]

origData = origData.drop(origData[origData.device_mac.isin(excluded_devices)].index )
origData

,time,device_mac,v0,v1,v2,v3,edit_date,edit_time
38,2021-08-28 09:00:00,21:fa:27:56,0.003342,0.942944,-0.278260,-0.181708,2021-08-28,09:00:00
39,2021-08-28 12:00:00,21:fa:27:56,0.003343,0.942954,-0.278225,-0.181710,2021-08-28,12:00:00
40,2021-08-28 15:00:00,21:fa:27:56,0.003342,0.942945,-0.278258,-0.181708,2021-08-28,15:00:00
46,2021-08-29 09:00:00,21:fa:27:56,0.003342,0.942944,-0.278260,-0.181708,2021-08-29,09:00:00
47,2021-08-29 12:00:00,21:fa:27:56,0.003342,0.942945,-0.278257,-0.181708,2021-08-29,12:00:00
...,...,...,...,...,...,...,...,...
85174,2021-09-04 12:00:00,ee:dd:4f:cd,0.999938,-0.004904,-0.004678,-0.000309,2021-09-04,12:00:00
85175,2021-09-04 15:00:00,ee:dd:4f:cd,0.000000,0.000000,0.000000,0.000000,2021-09-04,15:00:00
85181,2021-09-05 09:00:00,ee:dd:4f:cd,0.999938,-0.004904,-0.004679,-0.000309,2021-09-05,09:00:00
85182,2021-09-05 12:00:00,ee:dd:4f:cd,0.999938,-0.004904,-0.004680,-0.000309,2021-09-05,12:00:00


In [20]:
deviceLabels = pd.read_csv("/content/drive/MyDrive/DATA3001 Cyber/WeekendDeviceWithClusterLabelsHourly.csv", header = None, names = ['device_mac', 'cluster_label'])
#deviceLabels = deviceLabels.set_index(0)
deviceLabels

,device_mac,cluster_label
0,21:fa:27:56,0
1,21:fa:27:82,0
2,21:fa:29:50,0
3,21:fa:2d:cf,3
4,21:fa:2e:47,0
...,...,...
748,ee:dd:4e:5f,3
749,ee:dd:4e:60,4
750,ee:dd:4f:4c,3
751,ee:dd:4f:57,3


### Left joining tables to add cluster label column

In [32]:
#deviceLabels = deviceLabels.set_index('0')
origData = origData.merge(deviceLabels, how = 'left', on = 'device_mac')
origData.iloc[100]

time               2021-08-29 12:00:00
device_mac                 21:fa:37:f2
v0                         0.000178537
v1                            0.021779
v2                           0.0102947
v3                             0.20365
edit_date                   2021-08-29
edit_time                     12:00:00
cluster_label_x                      3
cluster_label_y                      3
cluster_label_x                      3
cluster_label_y                      3
cluster_label_x                      3
cluster_label_y                      3
cluster_label_x                      3
cluster_label_x                      3
cluster_label_y                      3
cluster_label                        3
Name: 100, dtype: object

In [33]:
cluster_data = origData.drop(['time', 'device_mac', 'edit_date', 'edit_time', 'cluster_label_x', 'cluster_label_y'], axis = 1)
cluster_data

,v0,v1,v2,v3,cluster_label
0,0.003342,0.942944,-0.278260,-0.181708,0
1,0.003343,0.942954,-0.278225,-0.181710,0
2,0.003342,0.942945,-0.278258,-0.181708,0
3,0.003342,0.942944,-0.278260,-0.181708,0
4,0.003342,0.942945,-0.278257,-0.181708,0
...,...,...,...,...,...
9031,0.999938,-0.004904,-0.004678,-0.000309,1
9032,0.000000,0.000000,0.000000,0.000000,1
9033,0.999938,-0.004904,-0.004679,-0.000309,1
9034,0.999938,-0.004904,-0.004680,-0.000309,1


In [34]:
cluster_data_np = cluster_data.to_numpy()
print(cluster_data_np.shape)
x_data_points = cluster_data_np[:,:-1]
print(x_data_points)
y_labels = cluster_data_np[:,-1]
print(y_labels)

(9036, 5)
[[ 3.34224347e-03  9.42944236e-01 -2.78260241e-01 -1.81707519e-01]
 [ 3.34259964e-03  9.42954119e-01 -2.78225092e-01 -1.81710057e-01]
 [ 3.34227101e-03  9.42944987e-01 -2.78257512e-01 -1.81707784e-01]
 ...
 [ 9.99938123e-01 -4.90417227e-03 -4.67948230e-03 -3.09096646e-04]
 [ 9.99938119e-01 -4.90435397e-03 -4.68013522e-03 -3.08931715e-04]
 [ 9.99938106e-01 -4.90498973e-03 -4.68227357e-03 -3.08999222e-04]]
[0. 0. 0. ... 1. 1. 1.]


## Using K-Neighbours Classifier with imported clusters
### data points are the 3 hour-aggregated flows

In [35]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [36]:
 X_train, X_test, y_train, y_test = train_test_split(x_data_points, y_labels, test_size=0.2, random_state=420)
 print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 print(X_test)

(7228, 4) (1808, 4) (7228,) (1808,)
[[ 3.34086351e-03  9.42911715e-01 -2.78401054e-01 -1.81660480e-01]
 [ 9.99953857e-01 -3.68126222e-03 -8.42488765e-04 -4.13845219e-04]
 [ 3.34085833e-03  9.42911026e-01 -2.78401181e-01 -1.81664771e-01]
 ...
 [ 3.35557534e-03  9.43313148e-01 -2.76943818e-01 -1.81803983e-01]
 [ 9.99907552e-01 -5.95988584e-03 -8.47409513e-03  2.49393843e-05]
 [ 3.35652142e-03  9.43339288e-01 -2.76850371e-01 -1.81810618e-01]]


In [37]:
def num_neighbors_param_test():
  for i in range(1, 50):
    KNmodel = KNeighborsClassifier(n_neighbors = i)
    KNmodel.fit(X_train, y_train)
    results = KNmodel.score(X_test, y_test)
    print(i, " : ", results)

In [38]:
num_neighbors_param_test()

1  :  0.5287610619469026
2  :  0.5564159292035398
3  :  0.5630530973451328
4  :  0.5663716814159292
5  :  0.5691371681415929
6  :  0.5741150442477876
7  :  0.5768805309734514
8  :  0.5862831858407079
9  :  0.5846238938053098
10  :  0.5912610619469026
11  :  0.5929203539823009
12  :  0.5951327433628318
13  :  0.599004424778761
14  :  0.6039823008849557
15  :  0.603429203539823
16  :  0.6012168141592921
17  :  0.6012168141592921
18  :  0.599004424778761
19  :  0.5923672566371682
20  :  0.5978982300884956
21  :  0.599004424778761
22  :  0.6006637168141593
23  :  0.5973451327433629
24  :  0.6023230088495575
25  :  0.5984513274336283
26  :  0.6012168141592921
27  :  0.5962389380530974
28  :  0.5940265486725663
29  :  0.5951327433628318
30  :  0.6017699115044248
31  :  0.5984513274336283
32  :  0.5984513274336283
33  :  0.599004424778761
34  :  0.5978982300884956
35  :  0.5962389380530974
36  :  0.6017699115044248
37  :  0.5984513274336283
38  :  0.5973451327433629
39  :  0.5984513274336283


### 9 neighbours chosen based on hyperparameter test

In [40]:
KNmodel = KNeighborsClassifier(n_neighbors = 14)
KNmodel.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=14, p=2,
                     weights='uniform')

In [41]:
results = KNmodel.score(X_train, y_train)
results

0.6372440509131156

In [42]:
res1 = KNmodel.predict(X_train)
res1

array([3., 2., 0., ..., 2., 2., 3.])

In [43]:
res2 = KNmodel.predict_proba(X_test)
res2.shape

(1808, 5)

### Get probability that data point belongs to its actual cluster according to model

### if probability is less than 10%, flag as potential anomaly

In [45]:
prob_of_correct = np.ones((X_test.shape[0],1))

for i in range(X_test.shape[0]):
  correct_cluster = int(y_test[i])
  prob_of_correct[i] = res2[i, correct_cluster]

In [46]:
print(prob_of_correct.shape[0])
total = prob_of_correct.shape[0]
print(prob_of_correct[prob_of_correct > 0.1].shape[0])
condition = prob_of_correct[prob_of_correct > 0.10].shape[0]

print(condition/total)

1808
1544
0.8539823008849557
